In [1]:
from theano.sandbox import cuda

In [23]:
%matplotlib inline
import os
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from keras import optimizers


In [3]:
#path = "data/ml-20m/"
path = "/Users/mdymshits/fastai/data/ml-latest-small/"
model_path = path + 'models'
if not os.path.exists(model_path):
    os.mkdir(model_path)
batch_size=64

## Set up data

We're working with the movielens data, which contains one rating per row, like this:

In [4]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [27]:
len(ratings)

100836

Just for display purposes, let's read in the movie names too.

In [5]:
pd.read_csv(path+'movies.csv').head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [30]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [31]:
movie_names

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (Cit\xc3\xa9 des enfants perdus, La) (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (a.k.a. 12 Monkey

In [6]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [7]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [8]:
movieid2idx

{1: 0,
 2: 481,
 3: 1,
 4: 482,
 5: 483,
 6: 2,
 7: 484,
 8: 485,
 9: 5017,
 10: 486,
 11: 487,
 12: 1399,
 13: 488,
 14: 3678,
 15: 489,
 16: 490,
 17: 491,
 18: 2657,
 19: 492,
 20: 3604,
 21: 291,
 22: 493,
 23: 2163,
 24: 494,
 25: 495,
 26: 496,
 27: 497,
 28: 2430,
 29: 2068,
 30: 6914,
 31: 259,
 32: 292,
 34: 460,
 36: 461,
 38: 1859,
 39: 462,
 40: 6704,
 41: 498,
 42: 5992,
 43: 499,
 44: 971,
 45: 293,
 46: 500,
 47: 3,
 48: 1400,
 65585: 4529,
 50: 4,
 52: 294,
 53: 3711,
 54: 501,
 55: 3679,
 57: 2651,
 58: 295,
 60: 502,
 61: 503,
 62: 504,
 63: 5832,
 64: 1401,
 65: 505,
 66: 506,
 68: 4035,
 69: 2663,
 70: 5,
 71: 4663,
 72: 5550,
 73: 4698,
 74: 2420,
 75: 5018,
 76: 507,
 77: 9348,
 78: 2421,
 79: 508,
 80: 6915,
 81: 4036,
 82: 2503,
 83: 5687,
 85: 4037,
 86: 509,
 87: 510,
 88: 511,
 89: 512,
 92: 513,
 93: 514,
 94: 2431,
 95: 515,
 96: 8152,
 97: 5289,
 99: 5551,
 100: 516,
 101: 6,
 102: 517,
 103: 8579,
 104: 518,
 105: 519,
 106: 296,
 107: 1402,
 108: 9349,
 

We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.

In [9]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [10]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [11]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247
2,0,2,4.0,964982224


In [12]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 609, 0, 9723)

In [13]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(610, 9724)

This is the number of latent factors in each embedding.

In [14]:
n_factors = 50

In [15]:
np.random.seed = 42

Randomly split into training and validation.

In [16]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

## Dot product

The most basic model is a dot product of a movie embedding and a user embedding. Let's see how well that works:

In [17]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [20]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.01), loss='mse')

In [24]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(optimizer='sgd', loss='mse')

In [25]:
model.optimizer.lr=0.1
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=8, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/2
80581/80581 [==============================] - 27s - loss: 3.2560 - val_loss: 3.2949
Epoch 2/2
80581/80581 [==============================] - 20s - loss: 2.4497 - val_loss: 3.0512


In [27]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=4, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/2
80581/80581 [==============================] - 40s - loss: 2.1025 - val_loss: 2.9898
Epoch 2/2
80581/80581 [==============================] - 40s - loss: 2.0269 - val_loss: 2.9673


The [best benchmarks](http://www.librec.net/example.html) are a bit over 0.9, so this model doesn't seem to be working that well...

##  Bias

The problem is likely to be that we don't have bias terms - that is, a single bias for each user and each movie representing how positive or negative each user is, and how good each movie is. We can add that easily by simply creating an embedding with one output for each movie and each user, and adding it to our output.

In [28]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [29]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [30]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [31]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [33]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(optimizer='sgd', loss='mse')
model.optimizer.lr=0.1

In [40]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=4, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/4
80581/80581 [==============================] - 2s - loss: 0.9816 - val_loss: 1.0111
Epoch 2/4
80581/80581 [==============================] - 2s - loss: 0.9471 - val_loss: 0.9826
Epoch 3/4
80581/80581 [==============================] - 2s - loss: 0.9182 - val_loss: 0.9579
Epoch 4/4
80581/80581 [==============================] - 2s - loss: 0.8934 - val_loss: 0.9372


In [41]:
model.optimizer.lr=0.01

In [42]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/3
80581/80581 [==============================] - 2s - loss: 0.8720 - val_loss: 0.9196
Epoch 2/3
80581/80581 [==============================] - 2s - loss: 0.8534 - val_loss: 0.9041
Epoch 3/3
80581/80581 [==============================] - 2s - loss: 0.8370 - val_loss: 0.8904


In [43]:
model.save_weights(model_path+'bias.h5')

In [44]:
model.load_weights(model_path+'bias.h5')

We can use the model to generate predictions by passing a pair of ints - a user id and a movie id. For instance, this predicts that user #3 would really enjoy movie #6.

In [45]:
model.predict([np.array([3]), np.array([6])])

array([[ 3.3798]], dtype=float32)

##  Neural net

Rather than creating a special purpose architecture (like our dot-product with bias earlier), it's often both easier and more accurate to use a standard neural network. Let's try it! Here, we simply concatenate the user and movie embeddings into a single vector, which we feed into the neural net.

In [49]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [52]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(optimizer='sgd', loss='mse')

In [53]:
model.optimizer.lr=0.01
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/2
80581/80581 [==============================] - 3s - loss: 1.3483 - val_loss: 1.0595
Epoch 2/2
80581/80581 [==============================] - 3s - loss: 1.0563 - val_loss: 0.9948


In [54]:
model.optimizer.lr=0.01
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/2
80581/80581 [==============================] - 3s - loss: 1.0071 - val_loss: 0.9598
Epoch 2/2
80581/80581 [==============================] - 3s - loss: 0.9763 - val_loss: 0.9353ETA: 0s - lo


In [56]:
model.optimizer.lr=0.05
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=16, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/2
80581/80581 [==============================] - 12s - loss: 0.8175 - val_loss: 0.8085s: 0.8 - ETA: 0s - loss: 
Epoch 2/2
80581/80581 [==============================] - 12s - loss: 0.8017 - val_loss: 0.7959


This improves on our already impressive accuracy even further!

## Withoud Dropout

In [64]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [65]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
# x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
# x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(optimizer='sgd', loss='mse')

In [66]:
model.optimizer.lr=0.1
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=16, nb_epoch=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/10
80581/80581 [==============================] - 11s - loss: 0.9943 - val_loss: 0.8800
Epoch 2/10
80581/80581 [==============================] - 10s - loss: 0.8348 - val_loss: 0.8531
Epoch 3/10
80581/80581 [==============================] - 10s - loss: 0.7858 - val_loss: 0.8165
Epoch 4/10
80581/80581 [==============================] - 9s - loss: 0.7591 - val_loss: 0.8086
Epoch 5/10
80581/80581 [==============================] - 9s - loss: 0.7429 - val_loss: 0.8087
Epoch 6/10
80581/80581 [==============================] - 9s - loss: 0.7298 - val_loss: 0.8031
Epoch 7/10
80581/80581 [==============================] - 9s - loss: 0.7197 - val_loss: 0.7971
Epoch 8/10
80581/80581 [==============================] - 9s - loss: 0.7096 - val_loss: 0.7981
Epoch 9/10
80581/80581 [==============================] - 10s - loss: 0.7000 - val_loss: 0.8108
Epoch 10/10
80581/80581 [==============================] - 9s - loss: 0.6912 - val_loss: 0.

In [59]:
model.optimizer.lr=0.1
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80581 samples, validate on 20255 samples
Epoch 1/2
80581/80581 [==============================] - 2s - loss: 0.7313 - val_loss: 0.8069
Epoch 2/2
80581/80581 [==============================] - 2s - loss: 0.7239 - val_loss: 0.8045
